30. Wat is het aantal pakketjes dat bezorgd is per maand?

In [ ]:
SELECT strftime('%Y-%m', sa_datum_distributiecollectie) AS maand, COUNT(*) AS aantal_pakketjes_bezorgd
FROM collo_packages
GROUP BY strftime('%Y-%m', sa_datum_distributiecollectie)
ORDER BY maand;

29. Welke maand heeft de kleinste hoeveelheid pakketjes die meteen de eerste keer in ontvangst zijn genomen? En wat is de hoeveelheid totaal bezorgde pakketjes voor deze maand?

In [ ]:
WITH eerste_aflevering_per_maand AS (
    SELECT strftime('%Y-%m', sa_datum_distributiecollectie) AS maand,
           COUNT(*) AS aantal_eerste_aflevering
    FROM collo_packages
    WHERE da_resultaatcode = '01-Gecollecteerd bij eerste poging' OR '01-Uitgereikt bij eerste uitreikpoging'
    GROUP BY strftime('%Y-%m', sa_datum_distributiecollectie)
),
kleinste_aantal_eerste_aflevering AS (
    SELECT maand, MIN(aantal_eerste_aflevering) AS kleinste_aantal
    FROM eerste_aflevering_per_maand
)
SELECT e.maand AS kleinste_maand, k.kleinste_aantal AS aantal_pakketjes
FROM kleinste_aantal_eerste_aflevering k
JOIN eerste_aflevering_per_maand e ON k.maand = e.maand;

28. Hoeveel pakketjes zijn op dezelfde dag geaccepteerd als gecollecteerd?

In [ ]:
SELECT COUNT(*) AS aantal_pakketjes
FROM collo_packages
WHERE DATE(da_datum_acceptatie) = DATE(da_datum_voormelding);

27. Welke pakketjes hebben een breedte groter dan 1000? En wat is hun lengte?

In [ ]:
SELECT dn_barcode, ma_lengte 
FROM collo_packages
WHERE ma_breedte > 1000;

26. Welke pakketjes hebben een gewicht groter dan 10000 gram en een breedte kleiner dan 300?

In [ ]:
SELECT dn_barcode 
FROM collo_packages
WHERE ma_gewicht > 10000 AND ma_breedte < 300;

25. Hoeveel pakketjes hebben een lengte groter dan 1000 en een breedte groter dan 500?

In [ ]:
SELECT dn_barcode
FROM collo_packages
WHERE ma_lengte > 1000 AND ma_breedte > 500;

24. Hoeveel pakketjes met een acceptatiecode in juli hebben resultaatgroepcode: “In process”?

In [ ]:
SELECT COUNT(*) AS aantal_pakketjes
FROM collo_packages
WHERE da_resultaatgroepcode = '03-In Process'
AND strftime('%Y-%m', da_datum_acceptatie) = '2023-07';

23. Welke unieke landcodes komen voor in de dataset?

In [ ]:
SELECT DISTINCT da_landcode_gea AS unieke_landcode
FROM collo_packages;

22. Hoeveel pakketjes hebben de resultaatgroepcode: “In process”?

In [ ]:
SELECT COUNT(*) FROM collo_packages
WHERE da_resultaatgroepcode = '03-In Process';

21. Hoeveel pakketjes zijn er gestuurd naar niet-particuliere adressen?

In [ ]:
SELECT COUNT(*) AS aantal_niet_particuliere_adress_paketjes
FROM collo_packages
WHERE da_type_adres_gea <> 'Particulier';

20. Wat is de minst voorkomende eindstatus van een pakket?

In [ ]:
SELECT da_resultaatcode AS minst_voorkomende_eindstatus, COUNT(*) AS aantal_voorkomen
FROM collo_packages
GROUP BY da_resultaatcode
ORDER BY aantal_voorkomen ASC
LIMIT 3;

19. Wat is het percentage pakketten dat bij de eerste bezorgpoging niet aan huis werd geleverd?

In [ ]:
SELECT (COUNT(CASE WHEN da_resultaatcode <> '01-Uitgereikt bij eerste uitreikpoging' THEN 1 END) * 100.0) / COUNT(*) AS percentage_niet_aan_huis_geleverd
FROM collo_packages;

18. Welke 10 postcodegebieden ontvingen de meeste pakketten en hoeveel?

In [ ]:
SELECT PC4_gea AS postcodegebied, COUNT(*) AS aantal_pakketten
FROM collo_packages
GROUP BY PC4_gea
ORDER BY aantal_pakketten DESC
LIMIT 10;

17. Wat is de gemiddelde tijd voor de reis van het sorteercentrum naar het distributiecentrum?

In [ ]:
SELECT AVG(JULIANDAY(sa_datum_distributiecollectie) - JULIANDAY(sa_dag_sortering1)) AS gemiddelde_tijd_in_dagen
FROM collo_packages
WHERE sa_datum_distributiecollectie IS NOT '9999-03-03'
AND sa_datum_sortering1 IS NOT '9999-03-03';

16. Welke dag van de week zag het hoogste volume aan pakketverzendingen?

In [ ]:
SELECT strftime('%w', sa_datum_distributiecollectie) AS dag_van_de_week,
       COUNT(*) AS aantal_verzendingen
FROM collo_packages
GROUP BY dag_van_de_week
ORDER BY aantal_verzendingen DESC
LIMIT 1;

In [ ]:
15. Hoeveel pakketten zijn er verzonden per maand, en hoe verhoudt dit zich tot het totaal in 2023?

In [ ]:
-- Aantal verzonden pakketten per maand in 2023
SELECT strftime('%m', sa_datum_distributiecollectie) AS maand,
       COUNT(*) AS aantal_pakketten
FROM collo_packages
WHERE strftime('%Y', sa_datum_distributiecollectie) = '2023'
GROUP BY maand;

-- Percentage van verzonden pakketten per maand vergeleken met totaal in 2023
SELECT strftime('%m', sa_datum_distributiecollectie) AS maand,
       COUNT(*) * 100.0 / (SELECT COUNT(*) FROM collo_packages WHERE strftime('%Y', sa_datum_distributiecollectie) = '2023') AS percentage_van_totaal
FROM collo_packages
WHERE strftime('%Y', sa_datum_distributiecollectie) = '2023'
GROUP BY maand;

14. Wat is de gemiddelde doorlooptijd van aanmelding tot bezorging van het pakket?

In [ ]:
SELECT AVG(JULIANDAY(da_datum_eindstatus) - JULIANDAY(da_datum_voormelding)) AS gemiddelde_doorlooptijd
FROM collo_packages
WHERE da_datum_eindstatus IS NOT '9999-03-03'
AND da_datum_voormelding IS NOT '9999-03-03';

In [ ]:
13. Hoeveel pakketten waren zwaarder dan 10 kg?

In [ ]:
SELECT COUNT(*) AS aantal_pakketjes
FROM collo_packages
WHERE ma_gewicht > 10000;

12. Hoeveel pakketten zijn er aangekomen in het distributiecentrum op 1 januari 2024?

In [ ]:
SELECT COUNT(*) AS aantal_pakketjes
FROM collo_packages
WHERE sa_datum_distributiecollectie = '2024-01-01';

11. Wat is de meest voorkomende bezorgvoorkeur onder klanten?

In [ ]:
SELECT deliverypreference AS meest_voorkomende_voorkeur, COUNT(*) AS aantal_voorkomen
FROM delivery_preference
GROUP BY deliverypreference
ORDER BY aantal_voorkomen DESC
LIMIT 3;

10. Wat is het aantal unieke account id’s die pakketen hebben ontvangen in het jaar 2023?

In [ ]:
SELECT COUNT(DISTINCT account_id_hashed) AS aantal_unieke_accounts
FROM collo_packages
WHERE strftime('%Y', da_datum_voormelding) = '2023';

9. Hoeveel accounts hebben hun gegevens voor het laatst bijgewerkt in 2023?

In [ ]:
SELECT COUNT(DISTINCT account_id_hashed) AS aantal_accounts_bijgewerkt
FROM delivery_preference
WHERE strftime('%Y', datelastupdated) = '2023';

8. Hoeveel pakketten werden er in december 2023 verzonden?

In [ ]:
SELECT COUNT(*) AS aantal_pakketten_dec
FROM collo_packages
WHERE strftime('%Y-%m', sa_datum_distributiecollectie) = '2023-12';

7. Hoeveel accounts zijn er gecreëerd in Juli?

In [ ]:
SELECT COUNT(DISTINCT delivery_preference.account_id_hashed) AS accounts_juli
FROM delivery_preference
WHERE strftime('%Y-%m', datecreated) = '2023-07';

6. Hoeveel pakketten werden er bezorgd in de maand Juli?

In [ ]:
SELECT COUNT(dn_barcode) AS aantal_pakketten_juli
FROM collo_packages
WHERE strftime('%Y-%m', da_datum_eindstatus) = '2023-07';

5. Hoeveel pakketten werden er bij de eerste bezorgpoging aan huis geleverd?

In [ ]:
SELECT COUNT(*) AS aantal_pakketten_eerste_bezorg
FROM collo_packages
WHERE da_resultaatcode IN ('01-Gecollecteerd bij eerste poging', '01-Uitgereikt bij eerste uitreikpoging');

4. Wat is de gemiddelde hoogte van de pakketten die in 2023 zijn verzonden?

In [ ]:
SELECT AVG(ma_hoogte) AS gemiddelde_hoogte_pakketen_in_2023
FROM collo_packages
WHERE strftime('%Y', da_datum_voormelding) = '2023';

3. Wat is het gemiddelde gewicht van de pakketten die in 2023 zijn verstuurd?

In [ ]:
SELECT AVG(ma_gewicht) AS gemiddelde_gewicht_pakketen_in_2023
FROM collo_packages
WHERE strftime('%Y', da_datum_voormelding) = '2023';

2. Wat is de gemiddelde breedte van de pakketten die zijn verzonden?

In [ ]:
SELECT AVG(ma_breedte) AS gemiddelde_breedte_pakketen
FROM collo_packages;

1. Hoeveel pakketten zijn er verstuurd in 2023?

In [ ]:
SELECT COUNT(*) AS aantal_verstuurd_pakketten
FROM collo_packages
WHERE strftime('%Y', sa_datum_distributiecollectie) = '2023';